In [1]:
from google.cloud.bigquery import *

In [2]:
import pandas as pd

In [3]:
from utils.bqutils import connect, init_catalog

In [4]:
client = connect('bqresearch')

In [5]:
catalog = init_catalog()

In [5]:
catalog.coviddatail.staging.covid19_by_age

coviddatail.staging.covid19_by_age

In [6]:
death_per_day = client.query('''
WITH total_deaths_per_date as
(select DATE(day) report_date, total
from staging.deaths_report_1
order by report_date

)
select * from total_deaths_per_date
'''
).to_dataframe()

In [8]:
catalog.coviddatail.stagin

In [8]:
population_per_year = client.query(f'''
select year, population
from `coviddatail.staging.population_table`
where age='total'
and sex='total'
and county='total'
and area='total'
order by year
'''
).to_dataframe()

In [9]:
population_per_year.year = population_per_year.year.astype(float)
population_per_year.population = population_per_year.population.astype(float)

In [10]:
from bokeh_frame import charts

In [11]:
import numpy as np

In [42]:
poly1 = np.polyfit(population_per_year.year+1, population_per_year.population, 1)
poly2 = np.polyfit(population_per_year.year+1, population_per_year.population, 2)

In [19]:
poly1

array([ 1.46403607e+05, -2.86533065e+08])

In [20]:
np.arange(2004, 2200, 10)

array([2004, 2014, 2024, 2034, 2044, 2054, 2064, 2074, 2084, 2094, 2104,
       2114, 2124, 2134, 2144, 2154, 2164, 2174, 2184, 2194])

In [63]:
population_per_year_fraction = population_per_year.set_index('year').reindex(np.arange(2000, 2023)).reset_index()

In [64]:
population_per_year_fraction

,year,population
0,2000,NaN
1,2001,6508817.0
2,2002,6631097.0
3,2003,6748402.0
4,2004,6869533.0
5,2005,6990722.0
6,2006,7116692.0
7,2007,7243634.0
8,2008,7419081.0
9,2009,7552049.0


In [65]:
population_per_year_fraction = population_per_year_fraction.join(pd.Series(np.arange(365)/365, name='year_fraction'), how='cross')
population_per_year_fraction['exact_year'] = population_per_year_fraction.year + population_per_year_fraction['year_fraction']
population_per_year_fraction['population'] = np.polyval(poly1, population_per_year_fraction['exact_year'])


In [70]:
population_per_year_fraction['exact_date'] = pd.to_datetime( year=population_per_year_fraction['year'], dayofyear=population_per_year_fraction['day'])

TypeError: to_datetime() got an unexpected keyword argument 'year'

In [19]:
population_per_year['month'] = 12
population_per_year['day'] = 31
population_per_year['date'] = pd.to_datetime(population_per_year.loc[:,['year', 'month', 'day']])
population_per_year['date_int'] = population_per_year['date'].view(int)
poly2 = np.polyfit(population_per_year.date_int, population_per_year.population, 2)
MN = population_per_year['date_int'].min()
MX = population_per_year['date_int'].max()
X =  np.arange(MN, MN + ((MX-MN) / 18)*(18+3),(MX-MN) / 18/365)
Y = np.polyval(poly2, X)
fr = pd.DataFrame({'exact_ts':X, 'population': Y})
fr['date'] = pd.to_datetime(fr.exact_ts)

In [23]:
fr.date.dt.date

0       2001-12-31
1       2002-01-01
2       2002-01-02
3       2002-01-03
4       2002-01-04
           ...    
7660    2022-12-25
7661    2022-12-26
7662    2022-12-27
7663    2022-12-28
7664    2022-12-29
Name: date, Length: 7665, dtype: object

In [20]:
charts.Dots(fr.date, fr.population).datetime().opts(width=1000)

Loading BokehJS ...

{'X': '@X', 'Y': '@Y'}


Figure(id='1004', ...)

In [99]:
pd.to_datetime(population_per_year['date_int'])

0    2001-12-31
1    2002-12-31
2    2003-12-31
3    2004-12-31
4    2005-12-31
5    2006-12-31
6    2007-12-31
7    2008-12-31
8    2009-12-31
9    2010-12-31
10   2011-12-31
11   2012-12-31
12   2013-12-31
13   2014-12-31
14   2015-12-31
15   2016-12-31
16   2017-12-31
17   2018-12-31
18   2019-12-31
Name: date_int, dtype: datetime64[ns]

In [68]:
(
    charts.Line('exact_year', 'population', population_per_year_fraction) +
    charts.Dots(np.arange(2000, 2024, 2), np.polyval(poly1, np.arange(2000, 2024, 2)), color='violet')
)

{'X': '@X', 'Y': '@Y', 'exact_year': '@exact_year', 'population': '@population'}


Figure(id='3298', ...)

In [32]:
death_per_day.total = death_per_day.total.astype(int)

In [21]:
death_per_day.report_date = pd.to_datetime(death_per_day.report_date)

In [16]:
from bokeh_frame import charts

In [33]:
death_per_day2 = death_per_day.set_index('report_date').resample('w').mean().reset_index()

In [34]:
charts.Line('report_date', 'total', death_per_day2).datetime().opts(width=1000)

{'report_date': '@report_date', 'total': '@total'}


Figure(id='2479', ...)

In [12]:
client.query('''
select * from 
staging.deaths_report_1
order by day desc
limit 1
''')

QueryJob<project=coviddatail, location=US, id=38293770-7cb1-4c68-b086-801fc1860eca>

In [13]:
a = _
a.to_dataframe()

,day,total,total_male,total_female,total_jewish,total_jewish_male,total_jewish_female,total_arab,total_arab_male,total_arab_female,...,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_89,age_90_plus
0,2021-11-28,116,55,61,101,45,56,15,10,5,...,0,3,4,5,7,15,13,20,21,25


In [ ]:
safe_run('''
